In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-mandarin/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

21056

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
with open(files[0]) as fopen:
    d = json.load(fopen)
    
d[0]

{'predict_zh': [50258,
  50260,
  50360,
  50365,
  21209,
  8225,
  1787,
  253,
  15106,
  8713,
  24302,
  10673,
  250,
  1369,
  100,
  30246,
  1546,
  18464,
  101,
  26748,
  50545,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'score_zh': 9.75,
 'filename': 'data_aishell/wav/train/S0002/BAC009S0002W0207.wav'}

In [10]:
tokenizer.decode(d[0]['predict_zh'], decode_with_timestamps = True)

'<|startoftranscript|><|zh|><|transcribe|><|0.00|>这样能够相对保障产品的质量<|3.60|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [11]:
from huggingface_hub import hf_hub_download

In [12]:
f = hf_hub_download(
    repo_id="mesolitica/pseudolabel-mandarin-large-v3-timestamp", 
    filename="translate-mandarin-ms.json",
    repo_type = 'dataset'
)

In [13]:
with open(f) as fopen:
    translation = json.load(fopen)

len(translation)

447949

In [14]:
len(files)

21056

In [15]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_zh'] > minimum_score:
                
                a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_zh']])}], 
                return_timestamps = True, return_language = 'zh', 
                time_precision = 0.02)[1]['chunks']
            
                a = [a_['text'] for a_ in a]
                
                t = ' |'.join(a)
                if t not in translation:
                    continue
                    
                t_translated = translation[t]
                t_translated_splitted = t_translated.split('|')
                if len(t_translated_splitted) != len(a):
                    continue
                
                mapping = {}
                for k in range(len(a)):
                    s = t_translated_splitted[k].strip()
                    if a[k][0] == ' ':
                        s = ' ' + s
                    if a[k][-1] == ' ':
                        s = s + ' '
                    mapping[a[k]] = s
            
                a = np.array(data[i]['predict_zh'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if t_ in mapping:
                            t_ = mapping[t_]
                        else:
                            t_ = ''
                    
                    splitted = t_.split()
                    if len(splitted):
                        ratio = (len(set(splitted)) / len(splitted))
                        if len(t_) > 100 and ratio < 0.5:
                            t_ = remove_duplicate(t_)
                    if len(t_) and t_[0] != ' ':
                        t_ = ' ' + t_
                            
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                    
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [16]:
results = loop((files[:10], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


In [17]:
len(results)

385

In [18]:
[r for r in results if r['new_text'].count('<|') > 6]

[{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Dia mendedahkan bahawa dia baru-baru ini melawat Sichuan sebagai duta<|5.58|><|5.58|> Penyakit mata<|7.74|><|7.74|> Empat hari<|8.94|><|endoftext|>',
  'audio_filename': 'data_aishell/wav/train/S0002/BAC009S0002W0412.wav'},
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Guan Yonghe Zibao pernah rancang bawa anak perempuannya bersamanya<|3.94|><|3.94|> Anak perempuannya sangat tua<|5.34|><|endoftext|>',
  'audio_filename': 'data_aishell/wav/train/S0002/BAC009S0002W0414.wav'},
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Saya lebih takut mati oleh serangga kosong<|3.44|><|3.44|> Persekitaran sangat mengejutkan<|4.80|><|endoftext|>',
  'audio_filename': 'data_aishell/wav/train/S0002/BAC009S0002W0418.wav'},
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Anak perempuan yang menonton filem berusia 39 tahun<|2.76|><|2.76|> Lebih tiga bulan<|4.06|><|endoftext|>',
  'audio_fi

In [19]:
results[100]['new_text'].count('<|')

6

In [ ]:
import IPython.display as ipd
ipd.Audio('data_aishell/wav/train/S0002/BAC009S0002W0212.wav')

In [21]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')
ori_tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|ms|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠDal',
 'am',
 'Ġera',
 'ĠInternet',
 'Ġini',
 '<|3.20|>',
 '<|endoftext|>']

In [43]:
results = mp.multiprocessing(files, loop, cores = 50)

In [24]:
len(results)

538102

In [25]:
results[-2]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Semakin ramai pendatang<|2.80|><|endoftext|>',
 'audio_filename': 'data_aishell/wav/train/S0002/BAC009S0002W0179.wav'}

In [26]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(results, test_size = 200)

In [27]:
len(train), len(test)

(537902, 200)

In [28]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [29]:
with open('prepared-mandarin-ms.jsonl', 'w') as fopen:
    for r in tqdm(train):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 537902/537902 [00:02<00:00, 267945.00it/s]


In [30]:
!ls -lh prepared-mandarin-ms.jsonl

-rw-r--r-- 1 ubuntu ubuntu 142M Apr 25 07:42 prepared-mandarin-ms.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
from huggingface_hub import HfApi
api = HfApi()

In [32]:
api.upload_file(
    path_or_fileobj='prepared-mandarin-ms.jsonl',
    path_in_repo='prepared-mandarin-ms.jsonl',
    repo_id='mesolitica/pseudolabel-mandarin-large-v3-timestamp',
    repo_type='dataset',
)

prepared-mandarin-ms.jsonl:   0%|          | 0.00/148M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-mandarin-large-v3-timestamp/blob/main/prepared-mandarin-ms.jsonl'

In [33]:
!head -n 10 prepared-mandarin-ms.jsonl

{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Berita Hiburan Sohu<|1.64|><|1.64|> Menurut laporan media Taiwan<|4.24|><|endoftext|>", "audio_filename": "data_aishell/wav/train/S0339/BAC009S0339W0422.wav"}
{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Ramalan cuaca Azure untuk minggu terkini<|3.58|><|endoftext|>", "audio_filename": "train/5_3102/5_3102_20170702195230.wav"}
{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Kedua-duanya didakwa sementara dengan satu pertuduhan membunuh<|2.92|><|endoftext|>", "audio_filename": "data_aishell/wav/train/S0221/BAC009S0221W0464.wav"}
{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Anda juga boleh menulisnya sendiri<|2.40|><|2.40|> Anda juga boleh mengarangnya sendiri<|3.50|><|endoftext|>", "audio_filename": "train/15_4546/15_4546_20170826121937.wav"}
{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Tolong berikan saya lagu oleh Xu Ruyun<|5.20|><|endoftext|>", "audi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
with open('test-mandarin-dataset-ms.json', 'w') as fopen:
    json.dump(test, fopen)

In [35]:
api.upload_file(
    path_or_fileobj='test-mandarin-dataset-ms.json',
    path_in_repo='zh-ms/test-mandarin-dataset-ms.json',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-mandarin-dataset-ms.json:   0%|          | 0.00/55.2k [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/zh-ms/test-mandarin-dataset-ms.json'

In [36]:
!rm -rf zh-ms
!mkdir zh-ms

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
import shutil

for t in test:
    f = os.path.split(t['audio_filename'])[1]
    shutil.copyfile(t['audio_filename'], os.path.join('zh-ms', f)) 
    
len(glob('zh-ms/*'))

200

In [41]:
# !sudo apt install zip -y

In [42]:
# !zip -r test-zh-ms-audio.zip zh-ms

In [40]:
api.upload_file(
    path_or_fileobj='test-zh-ms-audio.zip',
    path_in_repo='zh-ms/test-zh-ms-audio.zip',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-zh-ms-audio.zip:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/zh-ms/test-zh-ms-audio.zip'